# Offshore Wind to Hydrogen Supply Chain

### Framework For Case Studies: Single-Facility - Single-Demand Model

Importing Necessary Modules

In [ ]:
from pyomo.core import *
from pyomo.environ import *
import numpy as np
import pandas as pd

import math

import time
import matplotlib.pyplot as plt

Parameters & Sets used in Model:

In [ ]:
# Parameters

Times = list(np.arange(0,365))


# Demand

#WindTurbine Costs

PCWT =  3.00 #Annualized Capital Cost of 15MW Wind Turbine (M$)
POWT =  1.05 #Annual Maintenance Cost of 15MW Wind Turbine (M$)

#Desalination Costs

PCDes =  47*10**(-6)*1.35 #Annualized Capital Cost of Desalination per kt of h2 produced (M$)
PODes =  11.48*10**(-6)*1.35 #Annual Maintenance Cost of Desalination per kt of h2 produced (M$)

#Electrolyzer Costs

PCEL =  45.32*10**(0) #Annualized Capital Cost of Electrolyzer per GW of Capacity (M$)
POEL =  22.25*10**(0) #Annual Maintenance Cost of Desalination per GW of Capacity (M$)

#Liquefaction Costs

PCLiq =  1.5*627 #Annualized Capital Cost of Liquefaction per Capacity in kilotonnes per hour (M$) 8.148177*10*(3)
POLiq =  5*10**(-2)*PCLiq #Annual Maintenance Cost of Desalination per Capacity in kilotonnes per hour (M$)

#Storage Costs

PCStore =  3.667*10**(-1) #Annualized Capital Cost of Storage (M$) per kton
POStore =  9.0*10**(-2) #Annual Maintenance Cost of Storage (M$) per kton

#Ship Parameters

Shiptype = [0,1,2]
Shipsize = [1, 10, 14] #Ship capacity in kt of H2
Shipcost = [ 14256*10**(-6), 41930*10**(-6),33544*10**(-6)] #M$ per ship
transportcost = [60000*10**(-6),600000*10**(-6),840000*10**(-6)] #M$ per trip

# Floating Platform 

FP_Cost = (500/9.8)*(0.14)  #Annualized Capital Cost of Floating Platform (M$) per GW of Electrolyzer Capacity


In [ ]:
def Optimal_Design(State,CF,DemadValue):
    
    # Functions:
    
    # PWRA:
    
    def Piecewise_Reapproximate():
        Aliq.insert(0,model.Cliq())
        Aliq.sort()
        Bliq.insert(Aliq.index(model.Cliq()),model.Cliq()**0.7983)

        Astore.insert(0,model.Cstore())
        Astore.sort()
        Bstore.insert(Astore.index(model.Cstore()),model.Cstore()**0.673)

        model.con1 = Piecewise(model.C2liq,model.Cliq,
                          pw_pts=Aliq,
                          pw_constr_type='EQ',
                          f_rule=Bliq,
                          pw_repn='SOS2')

        model.con2 = Piecewise(model.C2store,model.Cstore,
                          pw_pts=Astore,
                          pw_constr_type='EQ',
                          f_rule=Bstore,
                          pw_repn='SOS2')
    
    def warmstarter():
        var_dict = {}
        for var in model.component_data_objects(Var, active=True, descend_into=True):
            try:
                var_dict[str(var)] =value(var)
            except ValueError:   
                pass
        return var_dict  
    
    
    model = ConcreteModel()

    #Decision Variables

    model.NWT = Var(initialize=1,within=Integers,bounds=(0,100000)) #No of Wind Turbines to be installed
    model.CDes = Var(initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Desalination Capacity to be installed in H2 kt/hr
    model.CEL = Var(initialize=11,within=NonNegativeReals,bounds=(0,10**6)) #Electrolyzer Capacity to be installed in MW

    model.Cliq = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,1000)) #Liquifaction Capacity to be installed in kTonnes per hr
    model.Cstore = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,10**3)) #LH2 Storage Capacity to be installed in kTonnes
    
    model.NS = Var(Shiptype,initialize=1,within=Integers,bounds=(0,365)) #No of Ships of each type to be purchased
    model.NT = Var(Shiptype,initialize=1,within=Integers,bounds=(0,10**6)) #No of Trips of each type purchased
    model.s = Var(Shiptype,Times,initialize=1,within=Integers,bounds = (0,365)) #No of shipments for each shiptype

    #PowerFlow Variables
    model.EWT = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**8)) #Power generated by Wind Turbines in each t
    model.EDes = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**8)) #Power consumed by Desalination in each t
    model.EEL = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**8)) #Power consumed by Electrolysis in each t
    model.Eliq = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**8)) #Power consumed by Liquefaction in each t

    #Hydrogen Flow/Level Variables
    model.Lstore = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #LH2 Level in the tank in each t in ktonnes
    model.F = Var(Times,initialize=50,within=NonNegativeReals,bounds=(0,10**6)) #H2 produced in each t in ktonnes/hr
    
    # Piece-Wise Linear Approxiamtion:
    Aliq = list(np.arange(0,1100,500))
    Bliq = []
    Bstore = []
    for i in range(len(Aliq)):
        Bliq.append(Aliq[i]**0.6791)

    Astore = list(np.arange(0,1100,500))  
    for i in range(len(Astore)):
        Bstore.append(Astore[i]**0.6733)

    model.C2liq = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,Bliq[-1]))    

    model.con1 = Piecewise(model.C2liq,model.Cliq,
                          pw_pts=Aliq,
                          pw_constr_type='EQ',
                          f_rule=Bliq,
                          pw_repn='SOS2')

    model.C2store = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,Bstore[-1]))
    model.con2 = Piecewise(model.C2store,model.Cstore,
                          pw_pts=Astore,
                          pw_constr_type='EQ',
                          f_rule=Bstore,
                          pw_repn='SOS2')
    print(Aliq)
    
    # Objective:
    def objrule(model):
        return (0
                +(PCWT + POWT)*model.NWT #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL #Capex & Opex Costs of Electrolyzer
               +(PCLiq*model.C2liq + POLiq*model.C2liq) #Capex & Opex Costs of Liquefaction
               +(PCStore*model.C2store + POStore*model.C2store) #Capex & Opex Costs of storage
               +sum(model.NS[i]*Shipcost[i] for i in Shiptype) #Capex Costs of Ships
               +sum(model.NT[i]*transportcost[i] for i in Shiptype) #Transportation Costs of Ships
               +FP_Cost*model.CEL #Capex Costs of Floating Platform
                )

    model.obj = Objective(rule=objrule, sense=minimize)
    
    #OOBJ:
    def oobj():
        val = ((0
                +(PCWT + POWT)*model.NWT() #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes() #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL() #Capex & Opex Costs of Electrolyzer
               +(PCLiq*model.Cliq()**0.7983 + POLiq*model.Cliq()**0.7983) #Capex & Opex Costs of Liquefaction
               +(PCStore*model.Cstore()**0.673 + POStore*model.Cstore()**0.673) #Capex & Opex Costs of storage
               +sum(model.NS[i]()*Shipcost[i] for i in Shiptype) #Capex Costs of Ships
               +sum(model.NT[i]()*transportcost[i] for i in Shiptype) #Transportation Costs of Ships
                +FP_Cost*model.CEL() #Capex Costs of Floating Platform
                ))
        return val

    
    # Constraints:
    
    # Energy Balance:
    
    model.energybalance = ConstraintList()
    for t in Times:
        model.energybalance.add(model.EWT[t] == model.EDes[t] + model.EEL[t] + model.Eliq[t])

    model.windpower = ConstraintList()
    for t in Times:
        model.windpower.add(model.EWT[t] <= CF[t]*0.015*24*model.NWT) #C1

    model.desalination = ConstraintList()
    for t in Times:
        model.desalination.add(model.EDes[t] == 52.5*10**(-3)*model.F[t]) #0.0494*10**(-6)

    model.electrolysis = ConstraintList()
    for t in Times:
        model.electrolysis.add(model.EEL[t] == model.F[t]*50*10**(0)) #check

    model.liquefaction = ConstraintList()
    for t in Times:
        model.liquefaction.add(model.Eliq[t] == (model.F[t])*33.3*10**(0)/(3)) # Check
    
    # Capacity
    
    model.c1 = ConstraintList()
    for t in Times:
        model.c1.add(model.F[t]/24 <= model.CDes) #Desalination Capacity

    model.c2 = ConstraintList()
    for t in Times:
        model.c2.add(model.EEL[t]/24 <= model.CEL) #Electrolyzer Capacity

    model.c3 = ConstraintList()
    for t in Times:
        model.c3.add(model.F[t]/24 +0.015*10**(-2)*model.Lstore[t] <=model.Cliq) #liquefaction Capacity
    
    # Transport & Inv:
    
    model.mb = ConstraintList()
    for t in Times:
        if t == 0:
            model.mb.add(model.Lstore[t] == (model.Lstore[364]+model.F[t]-sum(model.s[i,t]*Shipsize[i] for i in Shiptype)))
        else:
            model.mb.add(model.Lstore[t] == (model.Lstore[t-1]+model.F[t]-sum(model.s[i,t]*Shipsize[i] for i in Shiptype)))

    model.sc = ConstraintList()
    for i in Shiptype:
        for t in list(np.arange(0,363)):
            model.sc.add(sum(model.s[i,t] for t in list(np.arange(t,t+2)))<=model.NS[i])

    model.sc2 = ConstraintList()
    model.sc2.add(sum(model.s[i,t] for t in list([363,364]))<=model.NS[i])
    model.sc2.add(sum(model.s[i,t] for t in list([364,0]))<=model.NS[i])

    model.v2 = ConstraintList()
    for t in Times:
        model.v2.add(model.Lstore[t]<=model.Cstore)

    model.th = ConstraintList()
    for t in Times:
        if t == 0:
            model.th.add(sum(model.s[i,t]*Shipsize[i] for i in Shiptype)<=model.Lstore[364])
        else:
            model.th.add(sum(model.s[i,t]*Shipsize[i] for i in Shiptype)<=model.Lstore[t-1])

    model.other = ConstraintList()
    for i in Shiptype:
        model.other.add(model.NT[i] == sum(model.s[i,t] for t in Times))

    model.other2 = ConstraintList()
    for i in Shiptype:
        model.other2.add(model.NT[i] <=182*model.NS[i])

    model.demand = ConstraintList()
    model.demand.add(sum(model.F[t] for t in Times) >= 1)


    model.demand.add(sum(model.NT[i]for i in Shiptype)>=1 )

    model.ship = ConstraintList()
    for i in Shiptype:
        model.ship.add(sum(model.NT[i]*Shipsize[i] for i in Shiptype) >=DemandValue)
        
    #Solution Approach: Branch & Refine:
    
    tic = time.perf_counter()
    
    #change2 start
    q1=[]
    ub1=[]
    lb1=[]
    gap1=[]
    #change2 end

    solver = SolverFactory('gurobi')
    solver.options['mipgap'] = 0.01
    solver.options['MIPFocus'] = 1
    solver.options['Heuristics'] = 1
    solver.options['cuts']=3

    solver.options['ScaleFlag']=2

    #mip_start = warmstarter()
    solver.solve(model,tee=True)

    #model.display()
    LB = model.obj()
    UB = oobj()
    Gap = 1-(abs(LB)/abs(UB))

    print("LB=",LB)
    print("UB=",UB)
    print("Gap=",Gap)
    #Change Start                        
    #q1.append(q)
    ub1.append(UB)
    lb1.append(LB)
    gap1.append(Gap)
    #Change End

    while Gap>0.001:
    
        Piecewise_Reapproximate()

        model.del_component('model.obj')
        model.obj = Objective(rule=objrule, sense=minimize)

        mip_start = warmstarter()
        solver.solve(model,warmstart=mip_start,tee=True)
        #solver.solve(model,tee=True)

        LB = model.obj()
        UB = oobj()
        Gap = 1-(abs(LB)/abs(UB))
        print("LB=",LB)
        print("UB=",UB)
        print("Gap=",Gap)
        #Change Start                        
        #q1.append(q)
        ub1.append(UB)
        lb1.append(LB)
        gap1.append(Gap)
        #Change End
    toc = time.perf_counter()
    print(f"Completed in {toc - tic:0.4f} seconds")
    
      
    # Export variable values to Excel
#     variable_data = []
#     for var in model.component_data_objects(Var, active=True):
#         variable_data.append([var.name, var.value])

#     df = pd.DataFrame(variable_data, columns=['Variable', 'Value'])
#     df.to_excel('variable_values.xlsx', index=False)

    model_data = {'State': State}

    # Loop through each variable in the model
    for var in model.component_data_objects(Var, active=True):
        model_data[var.name] = var.value

    model_data['Objective'] = model.obj()
    model_data['Time'] = toc - tic
    model_data['Gap'] = Gap
    model_data['H2 Shipped'] = sum(model.NT[i]()*Shipsize[i] for i in Shiptype)
    model_data['LCOH'] = (model.obj()/sum(model.NT[i]()*Shipsize[i] for i in Shiptype))

    model_data['cost of wind'] = (PCWT + POWT)*model.NWT()
    model_data['cost of desalination'] = (PCDes + PODes)*model.CDes()
    model_data['cost of electrolyzer'] = (PCEL + POEL)*model.CEL()
    model_data['cost of liquefaction'] = (PCLiq*model.Cliq()**0.7983 + POLiq*model.Cliq()**0.7983)
    model_data['cost of storage'] = (PCStore*model.Cstore()**0.673 + POStore*model.Cstore()**0.673)
    model_data['cost of Transport'] = sum(model.NS[i]()*Shipcost[i] for i in Shiptype) + sum(model.NT[i]()*transportcost[i] for i in Shiptype)
    model_data['cost of floating platform'] = FP_Cost*model.CEL()

    model_data['Energy Generated'] = sum(model.EWT[t]() for t in Times)
    model_data['Energy for Desalination'] = sum(model.EDes[t]() for t in Times)
    model_data['Energy for Electrolysis'] = sum(model.EEL[t]() for t in Times)
    model_data['Energy for Liquefaction'] = sum(model.Eliq[t]() for t in Times)

    model_data['GHG of wind'] = (15/11)*model.NWT()*1.65e7
    model_data['GHG of desalination'] = sum(model.NT[i]()*Shipsize[i] for i in Shiptype)*(1000/2*18*20)*(2.25+4.85E-1)
    model_data['GHG of electrolyzer'] = model.CEL()*1000*5.85e4
    model_data['GHG of liquefaction'] = model.Cliq()*1000*24*1.15e5
    model_data['GHG of storage'] = model.Cstore()*1000*2.04e3
    model_data['GHG of Transport'] = sum(model.NT[i]()*model.NT[i]()*Shipsize[i] for i in Shiptype)*(1000*100*20)*(1.90e-2)

    # # Append the model data to the DataFrame
    ##results_df =  results_df.append(model_data, ignore_index=True)

    # # Save the DataFrame to Excel
    ##results_df.to_excel('case_study_results.xlsx', index=False)
    
    return model_data


In [ ]:
# open the excel file master_file as a pandas dataframe
master_file = pd.read_excel('master_file.xlsx', sheet_name='Sheet1')

results_df = pd.DataFrame(columns=['Model'])

# iterate through each row in the master_file dataframe
for index, row in master_file.iterrows():
    # extract the values from each row
    State =  row['Statename']
    DemandValue = row['Demand']
    buoys = row['Buoy']

    # read the file corresponding to buoys as excel from folder CF_data
    CF = pd.read_csv('CF_Data/'+str(buoys)+'.csv')

    CF = CF['CF'].tolist()

    # call the function
    print(State,buoys, DemandValue)

    model_data = Optimal_Design(State,CF,DemandValue)

   # Append the model data to the DataFrame
    results_df =  results_df.append(model_data, ignore_index=True)



In [ ]:
# Save the DataFrame to Excel
results_df.to_excel('Fullmodel_results_Statewise_LH2.xlsx', index=False)

final_results = results_df[['State','H2 Shipped','LCOH','Objective','cost of wind','cost of desalination','cost of electrolyzer','cost of liquefaction','cost of storage','cost of Transport','cost of floating platform','GHG of wind','GHG of desalination','GHG of electrolyzer','GHG of liquefaction','GHG of storage','GHG of Transport','Energy Generated','Energy for Desalination','Energy for Electrolysis','Energy for Liquefaction']]
final_results.to_excel('Statewise_LH2_Final_results.xlsx', index=False)
